In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vector_store = Chroma(
    embedding_function=OpenAIEmbeddings(model='text-embedding-3-large'),
    collection_name='real_estate_tax_collections',
    persist_directory='./real_estate_tax_collections'
)
retriever = vector_store.as_retriever(search_kwargs={'k': 3})

In [ ]:
from langchain_core.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever=retriever,
    name='real_estate_tax_retriever',
    description='Contains information about real estate tax up to December 2024'
)

In [ ]:
from langchain_tavily import TavilySearch

tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
    # include_answer=False,
    # include_raw_content=False,
    # include_images=False,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [ ]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_tool = ArxivQueryRun(api_wrapper=ArxivAPIWrapper(top_k_results=3))

In [ ]:
from langchain_google_community import GmailToolkit
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
SCOPES = [
    "https://www.googleapis.com/auth/gmail.compose",
    "https://www.googleapis.com/auth/gmail.send"
]

flow = InstalledAppFlow.from_client_secrets_file(
    "./google/gmail_credentials.json",
    SCOPES,
)
credentials = flow.run_local_server(port=0)
api_resource = build("gmail", "v1", credentials=credentials)
gmail_toolkit = GmailToolkit(api_resource=api_resource)
gmail_tool_list = gmail_toolkit.get_tools()

In [ ]:
tool_list = [retriever_tool, tavily_search_tool, arxiv_tool]
tool_list += gmail_tool_list

In [ ]:
from langgraph.prebuilt import ToolNode

tool_node = ToolNode(tool_list)
llm_with_tools = llm.bind_tools(tool_list)

In [ ]:
from typing import Literal
from langgraph.graph import MessagesState

class AgentState(MessagesState):
    summary: str
    

In [ ]:
from langgraph.types import interrupt, Command

def human_review(state: AgentState) -> Command[Literal['tools', 'agent']]:
    """
    'human_review' Node
    : LLM의 도구 호출에 대해 사용자의 검토를 요청한다.

    Args:
        - state(AgentState): 메시지 상태와 요약을 포함하는 state

    Returns:
        - Command[Literal['tools', 'agent']: 다음 node로 이동하기 위한 Command
    """
    
    messages = state['messages']
    last_ai_message = messages[-1]
    last_tool_call = last_ai_message.tool_calls[-1]
    
    # review
    human_review = interrupt(
        {
            'question': '이렇게 진행하면 될까요?',
            'tool_call': last_tool_call
        }
    )
    review_action = human_review['action']
    review_data = human_review.get('data', None)
    
    ## 에이전트의 판단이 맞다면, 도구를 사용하기 위해 아무것도 수정하지 않고 `tools` 노드로 이동
    if review_action == 'continue':
        return Command(goto='tools')
    
    ## 도구를 더 효율적으로 사용하기 위해 AIMessage의 `tool_calls` 필드를 업데이트
    if review_action == 'update_args':
        update_ai_message = {
            'id': last_ai_message.id,
            'role': 'ai',
            'content': last_ai_message.content,
            'tool_calls': [
                {
                    'id': last_tool_call['id'],
                    'name': last_tool_call['name'],
                    'args': review_data
                }
            ]
        }
        
        return Command(goto='tools', update={'messages': [update_ai_message]})
    
    # 다른 도구를 사용하기 위해 `ToolMessage`를 업데이트
    if review_action == 'update_tool':
        updated_tool_message = {
            'tool_call_id': last_tool_call['id'],
            'role': 'tool',
            'name': last_tool_call['name'],
            'content': review_data
        }
        
        return Command(goto='agent', update={'messages': [updated_tool_message]})
    
    

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

summarize_prompt = PromptTemplate.from_template(
    """
    아래의 대화 이력을 요약해주세요.
    만일 기존 요약 내용이 있다면, 해당 요약을 포함해 요약해주세요.
    
    [대화 이력]
    {messages}
    
    [기존 요약]
    {summary}
    """
)

def summarize(state: AgentState) -> AgentState:
    """
    'summarize' Node
    : 주어진 상태의 메시지를 요약한다.

    Args:
        - state(AgentState): 메시지 상태와 요약을 포함하는 state

    Returns:
        - AgentState: 응답 메시지를 포함하는 새로운 state
    """
    
    messages = state['messages']
    summary = state['summary']
    
    summarize_chain = summarize_prompt | llm | StrOutputParser()
    ai_message = summarize_chain.invoke({'messages': messages, 'summary': summary})
    
    return {'summary': ai_message}

In [ ]:
from langchain_core.messages import RemoveMessage

def delete(state: AgentState) -> AgentState:
    """
    'delete' Node
    : 주어진 상태에서 오래된 메시지를 삭제한다.

    Args:
        - state(AgentState): 메시지 상태와 요약을 포함하는 state

    Returns:
        - AgentState: 오래된 메시지가 삭제된 새로운 state
    """
    
    messages = state['messages']
    
    return {'messages': [RemoveMessage(id=message.id) for message in messages[:-3]]}

In [ ]:
from langchain_core.messages import SystemMessage

def agent(state: AgentState) -> AgentState:
    """
    'agent' Node
    : 주어진 상태에서 메시지를 가져와 LLM 및 도구를 사용하여 응답 메시지를 생성한다.

    Args:
        - state(AgentState): 메시지 상태와 요약을 포함하는 state

    Returns:
        - AgentState: 응답 메시지를 포함하는 새로운 state
    """
    
    messages = state['messages']
    summary = state['summary']
    
    if summary != '':
        messages = [SystemMessage(content=f'Here is the summary of the earlier conversation: {summary}')] + messages
    
    ai_message = llm_with_tools.invoke(messages)
    
    return {'messages': [ai_message]}

In [ ]:
def should_continue(state: AgentState) -> Literal['human_review', 'summarize']:
    """
    주어진 state에 따라 다음 단계로 진행할지 결정한다.

    Args:
        - state(AgentState): 메시지와 도구 호출 정보를 포함하는 state

    Returns:
        - Literal['human_review', 'summarize']: 다음 단계로 'human_review' 또는 'summarize'를 반환
    """

    messages = state['messages']
    last_ai_message = messages[-1]

    return 'human_review' if last_ai_message.tool_calls else 'summarize'

In [ ]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(AgentState)

# nodes
graph_builder.add_node('agent', agent)
graph_builder.add_node('tools', tool_node)
graph_builder.add_node(human_review)
graph_builder.add_node(summarize)
graph_builder.add_node(delete)

# edges
graph_builder.add_edge(START, 'agent')
graph_builder.add_conditional_edges(
    'agent',
    should_continue,
    ['human_review', 'summarize'] 
)
graph_builder.add_edge('tools', 'agent')
graph_builder.add_edge('summarize', 'delete')
graph_builder.add_edge('delete', END)

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

checkpointer = MemorySaver()
graph = graph_builder.compile(checkpointer=checkpointer)

In [ ]:
graph

In [ ]:
from langchain_core.messages import HumanMessage

query = "LLM Survey 논문의 내용을 검색해서 요약해주세요."
config = {
    'configurable': {
        'thread_id': 'summarize_paper'
    }
}

for chunk in graph.stream({'messages': [HumanMessage(query)], 'summary': ''}, config=config, stream_mode='values'):
    chunk['messages'][-1].pretty_print()

In [ ]:
iter = graph.stream(
    input=Command(resume={
        'action': 'update_args',
        'data': {'query': 'Large Language Model: A Survey'}
    }),
    config=config,
    stream_mode='updates'
)

for chunk in iter:
    print(chunk)

In [ ]:
iter = graph.stream(
    input=Command(resume={
        'action': 'update_tool',
        'data': 'arxiv가 아닌 Web으로 검색해주세요.'    
    }),
    config=config,
    stream_mode='updates'
)

for chunk in iter:
    print(chunk)

In [ ]:
graph.get_state(config).values['messages']

In [ ]:
graph.get_state(config).values['summary']

In [ ]:
iter = graph.stream(
    input=Command(resume={'action': 'continue'}),
    config=config,
    stream_mode='updates'
)

for chunk in iter:
    print(chunk)

In [ ]:
graph.get_state(config).values['summary']